In [ ]:
import pandas as pd
from datetime import datetime
import re
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from time import sleep
from mongo_wrapper.mongo_wrapper import MongoWrapper
import os
from dotenv import load_dotenv
from mongo_wrapper.mongo_wrapper import MongoWrapper

In [2]:
MONGO_IP = os.getenv("MONGO_IP")
MONGO_PORT = os.getenv("MONGO_PORT")
MONGO_DB = os.getenv("MONGO_DB")
MONGO_USERNAME = os.getenv("MONGO_USERNAME")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")

In [4]:
mongo_client = MongoWrapper(
    db=MONGO_DB,
    user=MONGO_USERNAME,
    password=MONGO_PASSWORD,
    ip=MONGO_IP,
    port=MONGO_PORT
)

2025-05-01 16:10:56,104 WARNING Logger Mongo was configured with True console stream
2025-05-01 16:10:56,450 INFO Connected to thesis database on 206.81.16.39


In [5]:
channels = mongo_client.get_all_collections()

print(f'All channels: {channels}')

All channels: ['Michoacán empleo', 'Colima servicios básicos', 'proceso_unofficial', 'Baja California hambre', 'Colima pobreza', 'Tamaulipas alimentación', 'Sinaloa sociedad', 'Guerrero médico', 'Sinaloa integración', 'Guerrero crisis', 'Guanajuato médico', 'Zacatecas economía', 'Guanajuato seguro social', 'Guanajuato inseguridad', 'Guerrero economía', 'Guanajuato desarrollo', 'Colima habitaciónes', 'Michoacán atención médica', 'Guanajuato comunidad', 'Chihuahua alimentos', 'Jalisco habitaciónes', 'Sinaloa habitaciónes', 'Zacatecas salud', 'Tamaulipas hospital', 'Switzerland living', 'Chihuahua seguro social', 'Michoacán', 'Michoacán universidad', 'Switzerland food', 'elpaismexico', 'Chihuahua seguridad social', 'Guanajuato aprendizaje', 'Sinaloa crisis', 'Guanajuato pensión', 'Tamaulipas salud', 'Chihuahua integración', 'Baja California comunidad', 'Guanajuato condiciones', 'Switzerland employment', 'Tamaulipas vivienda', 'Chihuahua salud', 'Baja California', 'Zacatecas', 'politicomx'

In [6]:
channel_posts = {}
for channel in channels:
    posts = mongo_client.get_collection_entries(collection=channel)
    for post in posts:
        post.pop("comments", None)

    channel_posts[channel] = pd.DataFrame(posts)

In [14]:
channel_posts["elpaismexico"]

,_id,text,author,posting_ts
0,4306,"🔴 El ‘influencer’ ‘Fofo’ Márquez, un “niño ric...",-1001355178219,1.738202e+09
1,4305,🔴ÚLTIMA HORA | Trump choca con Petro: Estados ...,-1001355178219,1.737913e+09
2,4304,"🔴 La resolución, dictada por un tribunal del...",-1001355178219,1.737770e+09
3,4303,🇺🇸🔴 ÚLTIMA HORA | Donald Trump se convierte po...,-1001355178219,1.737393e+09
4,4302,🔴EN DIRECTO | “Mañana al mediodía se baja la p...,-1001355178219,1.737327e+09
...,...,...,...,...
1745,2968,🔴 ÚLTIMA HORA | El Tribunal Electoral permite ...,-1001355178219,1.682569e+09
1746,2967,"🔴 ÚLTIMA HORA | López Obrador, sobre su salud:...",-1001355178219,1.682553e+09
1747,2966,🔴 Adán Augusto López estima que López Obrador ...,-1001355178219,1.682551e+09
1748,2965,🔴 #Exclusiva | El expresidente de México Enriq...,-1001355178219,1.682517e+09


In [15]:
channel_posts["ElUniversalOnline"]

,_id,text,author,posting_ts
0,3236,#BajoReservaExprés Director del ISSSTE se trat...,-1001478122370,1.643565e+09
1,3235,Las mejores plumas de opinión ✒️ \n\nRoberto ...,-1001478122370,1.643556e+09
2,3234,#Bajo Reserva ✍️\n\n🤫 El silencio sobre José R...,-1001478122370,1.643553e+09
3,3233,"¡Bonito domingo, lectores!☀️🤓 Te presentamos l...",-1001478122370,1.643551e+09
4,3232,🤚¡Espera! Estas son las noticias más destacada...,-1001478122370,1.643518e+09
...,...,...,...,...
2430,5507,#BajoReservaExprés En el Senado no hay espacio...,-1001478122370,1.682536e+09
2431,5506,Las mejores plumas de opinión 🖊 \n \nCarlos L...,-1001478122370,1.682520e+09
2432,5505,🔰 #BajoReserva \n\n🔴 Truenan senadores contra...,-1001478122370,1.682519e+09
2433,5504,¡Excelente día! Te compartimos las notas de po...,-1001478122370,1.682517e+09


In [16]:
channel_posts["proceso_unofficial"]

,_id,text,author,posting_ts
0,911,"Consenso de ONG: El estado, principal agresor ...",-1001304238369,1.643569e+09
1,910,"Caso UDLAP: Al descubierto, la trama de Barbos...",-1001304238369,1.643569e+09
2,998,Ni con amparos acceden los niños a la vacuna\n...,-1001304238369,1.646146e+09
3,997,"Veracruz: ultraje a la autoridad, ""delito"" que...",-1001304238369,1.646096e+09
4,996,"Económico, el contrataque de Washington y sus ...",-1001304238369,1.646056e+09
...,...,...,...,...
3136,2096,La diplomacia tras bambalinas\nhttps://ift.tt/...,-1001304238369,1.680051e+09
3137,2095,La dimensión geopolítica del fentanilo: México...,-1001304238369,1.680008e+09
3138,2094,Renovación del INE: Juego de vencidas entre di...,-1001304238369,1.679965e+09
3139,2093,"Conflicto entre Cemex y Calica, pretexto ideal...",-1001304238369,1.679922e+09


In [17]:
channel_posts["politicomx"]

,_id,text,author,posting_ts
0,2918,🚨LO ÚLTIMO\n\n🔵Santiago Creel confía en que go...,-1001408283237,1.643586e+09
1,2917,🚨LO ÚLTIMO\n\n🔵Monreal afirma que Rocío Nahle ...,-1001408283237,1.643584e+09
2,2916,🚨LO ÚLTIMO\n\n🔵Rosa Icela Rodríguez destaca im...,-1001408283237,1.643583e+09
3,2915,🚨LO ÚLTIMO\n\n🔵Mesa de diálogo entre Segob y P...,-1001408283237,1.643558e+09
4,2914,🚨LO ÚLTIMO\n\n🔵PAN rechaza declaraciones de Ad...,-1001408283237,1.643558e+09
...,...,...,...,...
5098,7533,📌 AMLO dice que ministro Laynez se ‘extralimit...,-1001408283237,1.679932e+09
5099,7532,🔴 Esto opina AMLO sobre la experiencia de Bert...,-1001408283237,1.679929e+09
5100,7531,🔴 Mier acusa ‘activismo judicial’ del ministro...,-1001408283237,1.679889e+09
5101,7530,"⭕️ ‘Morena quiere consejeros a modo’, acusa op...",-1001408283237,1.679883e+09


In [18]:
channel_posts["lajornada_unofficial"]

,_id,text,author,posting_ts
0,12443,Editorial: Vacunas: camino correcto\nhttps://w...,-1001388777533,1.643544e+09
1,12442,Luis Pérez trabaja para consolidar la identida...,-1001388777533,1.643544e+09
2,12441,Llamado de auxilio por desaparecidos\nhttps://...,-1001388777533,1.643544e+09
3,12440,La FBI también adquirió el software espía Pega...,-1001388777533,1.643544e+09
4,12439,Final de la versión china de El club de la pel...,-1001388777533,1.643544e+09
...,...,...,...,...
18668,28783,Lidera México conflictos mineros en AL: 58 en ...,-1001388777533,1.679915e+09
18669,28782,Agreden a tepoztecos que se resisten a la inva...,-1001388777533,1.679915e+09
18670,28781,En el país hay una crisis de endeudamiento y a...,-1001388777533,1.679915e+09
18671,28780,Manuel Pérez Rocha L.*: Andanada de demandas d...,-1001388777533,1.679915e+09


In [19]:
channel_posts["larazondemexico"]

,_id,text,author,posting_ts
0,7050,🚨 #ÚLTIMAHORA | Los Bengals derrotan a los Chi...,-1001436192005,1.643585e+09
1,7049,¡Buen domingo! Esto es lo que tienes que saber...,-1001436192005,1.643560e+09
2,7048,🚨 #ÚLTIMAHORA | ¡Histórico! Nadal se impone a ...,-1001436192005,1.643552e+09
3,7047,📰 ¡Buenas noches! Estas son las noticias que t...,-1001436192005,1.643509e+09
4,7046,🚨 #ÚLTIMAHORA | México registra 522 defuncione...,-1001436192005,1.643498e+09
...,...,...,...,...
4243,11173,De nuestra sección de opinión y análisis\n\nNo...,-1001436192005,1.679927e+09
4244,11172,¡Buenos días! Compartimos contigo las noticias...,-1001436192005,1.679920e+09
4245,11170,🚨 #ÚLTIMAHORA | México rescata un sufrido empa...,-1001436192005,1.679884e+09
4246,11169,📰 ¡Buenas noches! Estas son las noticias que t...,-1001436192005,1.679882e+09


In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from sentence_transformers import SentenceTransformer, util
import torch
from dotenv import load_dotenv
from mongo_wrapper.mongo_wrapper import MongoWrapper

# states to categorize
STATES = [
    "Guanajuato",
    "Michoacán",
    "Sinaloa",
    "Chihuahua",
    "Guerrero",
    "Tamaulipas",
    "Baja California",
    "Zacatecas",
    "Colima",
    "Jalisco"]

# channels to analyze 
TARGET_CHANNELS = [
    "elpaismexico",
    "ElUniversalOnline",
    "proceso_unofficial",
    "politicomx",
    "lajornada_unofficial",
    "larazondemexico"]

# define dimensions of poverty 
POVERTY_DIMENSIONS = {
    "INCOME": """
    desempleo, sueldo mínimo, salario bajo, inflación, deudas, crisis, préstamos, 
    despidos, recortes, quiebra, pobreza, falta de chamba, pérdida de empleo, 
    no hay trabajo, sin chamba, unemployment, low wages, bankrupt, jobless,
    struggling, salario miserable, jodido, quedarse sin trabajo, desempleados,
    sin dinero, poco dinero, economía informal, ingreso insuficiente, 
    crisis económica, recesión, inflación alta, aumento de precios, 
    dificultades económicas, no alcanza, dinero no alcanza, falta de oportunidades,
    salario precario, empleo temporal, trabajo mal pagado, subempleo, 
    desigualdad económica, bajos ingresos, pobreza extrema, vivir al día,
    economía familiar, sustento familiar, ganar poco, ahorros, sin ahorros,
    trabajo informal, trabajo precario, costo de vida, gastos mensuales,
    pagar deudas, dificultad para pagar, impuestos altos, desigualdad salarial
    """,
    
    "ACCESS TO HEALTH SERVICES": """
    sin medicinas, hospital lejano, esperas, sin seguro, mala atención, enfermedad, 
    rechazado, sin tratamiento, medicinas caras, sin doctores, ir al doctor, 
    seguro médico, doctor particular, ir a consulta, healthcare, medical treatment, 
    doctor appointment, health insurance, sistema de salud deficiente, falta de médicos,
    hospitales saturados, emergencias médicas, atención médica, servicios médicos, 
    clínicas rurales, falta de especialistas, atención sanitaria, centros de salud,
    sistema de salud pública, citas médicas, listas de espera, tratamientos costosos,
    seguro popular, medicamentos, farmacias, problemas de salud, salud pública,
    enfermedades crónicas, falta de hospitales, ambulancias, atención primaria,
    pacientes, enfermos, medicina preventiva, salubridad, sistema sanitario,
    atención médica deficiente, cobertura médica, sin acceso a medicamentos,
    sin posibilidad de tratamiento, colapso hospitalario, saturación médica,
    falta de camas, sin quirófanos, desabastecimiento de medicinas, crisis sanitaria
    """,
    
    "EDUCATIONAL LAG": """
    sin escuela, analfabetismo, deserción, acceso a la educación, calidad educativa, 
    recursos didácticos, infraestructura escolar, escuela lejana, escuela lejos, 
    sin útiles, ausentismo, sin maestros, rezago escolar, sacar buenas notas, 
    education, school, learning, degree, student loans, abandono escolar,
    bachillerato incompleto, primaria incompleta, baja escolaridad, educación deficiente,
    maestros mal pagados, escuelas sin recursos, sin materiales escolares, 
    educación de baja calidad, fracaso escolar, repetir curso, repetir año,
    oportunidades educativas, igualdad educativa, acceso a la universidad,
    escuelas rurales, escuelas remotas, transporte escolar, oportunidades de estudio,
    sin computadoras, sin internet, brecha digital, educación a distancia,
    formación profesional, capacitación, nivel educativo, sistema educativo,
    reforma educativa, educación pública, sin educación, alfabetización,
    rendimiento académico, evaluar la educación, pruebas estandarizadas,
    desigualdad educativa, educación gratuita, colegiaturas, becas escolares,
    libros de texto, aprendizaje, estudiantes vulnerables, calidad de enseñanza
    """,
    
    "ACCESS TO SOCIAL SECURITY": """
    sin contrato, economía informal, informal, sin pensión, sin derechos, explotación, 
    sin ahorro, sin prestaciones, desprotección, trabajo ilegal, sin seguro, 
    IMSS, ISSSTE, social security, worker rights, informal job, desprotección social,
    sin beneficios laborales, trabajo sin contrato, empleo informal, 
    sin cotizar, sin jubilación, sistema de pensiones, fondos de pensiones,
    seguridad social deficiente, derechos laborales, protección laboral,
    prestaciones sociales, trabajo en negro, trabajo sin seguridad social,
    precariedad laboral, aportes sociales, contribuciones, ahorro para el retiro,
    sistema de jubilación, pensión digna, caja de ahorros, prestaciones,
    sindicatos, contratos temporales, contratos precarios, trabajadores vulnerables,
    indemnización, despido, antigüedad laboral, protección al trabajador,
    subsidios, apoyo social, protección social, beneficios sociales,
    beneficios gubernamentales, programas de apoyo, programas sociales
    """,
    
    "HOUSING": """
    sin agua, sin luz, hacinamiento, desalojo, vivienda precaria, sin techo, 
    goteras, renta cara, casa insegura, sin baño, techos de lámina, cuartos de cartón, 
    casas abandonadas, inundaciones, bad housing, slum, no electricity, unsafe home, 
    eviction notice, vivir entre ratas, se les cae la casa, vivienda inadecuada,
    condiciones insalubres, viviendas improvisadas, barrios marginales, favelas,
    asentamientos irregulares, colonias populares, zonas vulnerables, viviendas pequeñas,
    sin ventilación, sin servicios básicos, sin drenaje, sin alcantarillado,
    sin calefacción, sin gas, sin internet, sin infraestructura, vivienda indigna,
    vivienda social, hipotecas, créditos inmobiliarios, desalojos forzosos,
    ocupación ilegal, viviendas informales, tugurios, chabolas, materiales de construcción,
    construcción deficiente, servicios públicos, electricidad irregular, agua potable,
    agua contaminada, costo de vivienda, alquiler elevado, mercado inmobiliario,
    acceso a vivienda, propiedad de la vivienda, sin escrituras, terrenos irregulares
    """,
    
    "ACCESS TO FOOD": """
    hambre, desnutrición, comida escasa, sin alimentos, comida cara, ayuda alimentaria, 
    escasez, comida mala, dieta pobre, inseguridad alimentaria, canasta básica, 
    food security, nutrition, meal, groceries, comida suficiente, alimentos nutritivos,
    acceso a alimentos, alimentos frescos, alimentos saludables, nutrición infantil,
    desnutrición crónica, malnutrición, hambruna, bancos de alimentos, comedores sociales,
    comedores populares, comedores comunitarios, programas alimentarios, despensas,
    ayuda alimenticia, dieta equilibrada, alimentación básica, costo de alimentos,
    precios altos, suministro de alimentos, abastecimiento, mercados locales,
    soberanía alimentaria, producción de alimentos, agricultura local, huertos urbanos,
    consumo de proteínas, frutas y verduras, cereales, granos básicos, tortillas,
    frijoles, arroz, maíz, trigo, leche, carne, productos básicos, canasta alimentaria,
    seguridad alimentaria, crisis alimentaria, ayuno, hambre infantil, hambre crónica,
    falta de comida, no poder comer, saltarse comidas, comer una vez al día, ayuno forzado
    """,
    
    "SOCIAL COHESION": """
    fragmentación, polarización, exclusión, discriminación, conflicto, desconfianza, 
    marginalización, tensiones, estigmatización, diversidad, solidaridad, inclusión, 
    pertenencia, redes sociales, confianza, barrio, raza, community, belonging, inclusion,
    tejido social, capital social, participación comunitaria, segregación, exclusión social,
    grupos vulnerables, minorías, sentido de comunidad, cohesión comunitaria, 
    acción colectiva, organización vecinal, relaciones sociales, vínculos comunitarios,
    identidad colectiva, integración, desintegración social, aislamiento social,
    marginación, inequidad, injusticia social, racismo, clasismo, xenofobia,
    discriminación étnica, discriminación racial, pueblos indígenas, afrodescendientes,
    migrantes, desplazados, refugiados, derechos humanos, dignidad, respeto, tolerancia,
    convivencia, comunidad, apoyo mutuo, lazos comunitarios, movilización social,
    acción colectiva, autogestión, cooperación, redes de apoyo, organizaciones sociales
    """
}

class PovertyDimensionClassifier:
    def __init__(self):
        # sentence embedding
        self.model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
        
        # create embeddings for poverty dimensions
        self.dimension_names = list(POVERTY_DIMENSIONS.keys())
        self.dimension_embeddings = self.model.encode(list(POVERTY_DIMENSIONS.values()), convert_to_tensor=True)
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        text = re.sub(r'<.*?>', ' ', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ]', ' ', text)
        return re.sub(r'\s+', ' ', text).strip().lower()
    
    # classify the text into a poverty dimension using sentence embeddings
    def classify_text(self, text, threshold=0.10):
        if not text:
            return None, 0.0
        
        cleaned_text = self.clean_text(text)
        if len(cleaned_text) < 10:  # avoid too short texts
            return None, 0.0
        
        text_embedding = self.model.encode(cleaned_text, convert_to_tensor=True)
        
        # compute cosine similarity
        cosine_scores = util.cos_sim(text_embedding, self.dimension_embeddings)[0]
        
        # find dimension with highest similarity 
        max_idx = torch.argmax(cosine_scores).item()
        max_score = cosine_scores[max_idx].item()
        if max_score >= threshold:
            return self.dimension_names[max_idx], max_score
        else:
            return None, max_score

# load only data of interest from MongoDB
def load_state_posts():
    
    MONGO_IP = os.getenv("MONGO_IP")
    MONGO_PORT = os.getenv("MONGO_PORT")
    MONGO_DB = os.getenv("MONGO_DB")
    MONGO_USERNAME = os.getenv("MONGO_USERNAME")
    MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
    
    mongo_client = MongoWrapper(
        db=MONGO_DB,
        user=MONGO_USERNAME,
        password=MONGO_PASSWORD,
        ip=MONGO_IP,
        port=MONGO_PORT)
    
    all_channels = mongo_client.get_all_collections()
    available_target_channels = [channel for channel in TARGET_CHANNELS if channel in all_channels]
    
    # initialize a dictionary to store posts for each state
    state_posts = {state: [] for state in STATES}
    
    # classify posts by using regex patterns - if a post contains a state name, it will be classified as that state
    state_patterns = {state: re.compile(r'\b' + re.escape(state) + r'\b', re.IGNORECASE) for state in STATES}
    
    for channel in tqdm(available_target_channels, desc="loading channels"):
        posts = mongo_client.get_collection_entries(collection=channel)
        
        print(f"channel: {channel} - {len(posts)} post found")
        
        for post in tqdm(posts, desc=f"analysis {channel}", leave=False):
            post_text = post.get('text', '')
                
            for state, pattern in state_patterns.items():
                if pattern.search(post_text):
                    state_posts[state].append({
                        'text': post_text,
                        'author': post.get('author', ''),
                        'posting_ts': post.get('posting_ts', ''),
                        'channel': channel})
    
    for state in STATES:
        state_posts[state] = pd.DataFrame(state_posts[state])
    
    return state_posts

# classify posts by dimensions of poverty 
def analyze_poverty_dimensions(state_posts):
    classifier = PovertyDimensionClassifier()
    
    results = []

    for state, df in state_posts.items():
        print(f"\nanalyzing {state} ({len(df)} posts)...")
    
        dimension_counts = {dim: 0 for dim in POVERTY_DIMENSIONS.keys()}
        dimension_counts["OTHER"] = 0  # other types of posts 
    
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Classificazione {state}"):
            text = row['text']
        
            dimension, score = classifier.classify_text(text)
        
            if dimension:
                dimension_counts[dimension] += 1
            else:
                dimension_counts["OTHER"] += 1
        
        total_posts = len(df)
        dimension_percentages = {dim: (count / total_posts) * 100 for dim, count in dimension_counts.items()}
        
        print(f"\nresults for {state}:")
        print(f"total posts: {total_posts}")
        print("\ndistribution of posts across dimensions of poverty:")
        
        for dim, count in dimension_counts.items():
            dim_name = dim if dim != "OTHER" else "non-poverty posts"
            pct = dimension_percentages[dim]
            print(f"- {dim_name}: {count} post ({pct:.1f}%)")
        
        for dim in list(POVERTY_DIMENSIONS.keys()) + ["OTHER"]:
            results.append({
                'state': state,
                'dimension': dim,
                'count': dimension_counts[dim],
                'percentage': dimension_percentages[dim],
                'total_posts': total_posts})
    
    results_df = pd.DataFrame(results)
    return results_df

def main():
    state_posts = load_state_posts()
    
    results = analyze_poverty_dimensions(state_posts)
    
    results.to_csv("telegram_results.csv", index=False)
    
    pivot_counts = results.pivot(index='state', columns='dimension', values='count')
    print("\ncount of posts per dimension:")
    print(pivot_counts)
    
    pivot_percentages = results.pivot(index='state', columns='dimension', values='percentage')
    print("\npercentage of posts per dimension:")
    print(pivot_percentages.round(1))

if __name__ == "__main__":
    main()

2025-05-01 17:01:21,201 WARNING Logger Mongo was configured with True console stream
2025-05-01 17:01:21,558 INFO Connected to thesis database on 206.81.16.39
loading channels:   0%|          | 0/6 [00:00<?, ?it/s]

channel: elpaismexico - 1750 post found


loading channels:  17%|█▋        | 1/6 [00:00<00:02,  1.81it/s]

channel: ElUniversalOnline - 2435 post found


loading channels:  33%|███▎      | 2/6 [00:01<00:03,  1.14it/s]

...


results for Zacatecas:
total posts: 221

distribution of posts across dimensions of poverty:
- INCOME: 19 post (8.6%)
- ACCESS TO HEALTH SERVICES: 12 post (5.4%)
- EDUCATIONAL LAG: 23 post (10.4%)
- ACCESS TO SOCIAL SECURITY: 40 post (18.1%)
- HOUSING: 118 post (53.4%)
- ACCESS TO FOOD: 7 post (3.2%)
- SOCIAL COHESION: 2 post (0.9%)
- non-poverty posts: 0 post (0.0%)

analyzing Colima (108 posts)...


Classificazione Colima: 100%|██████████| 108/108 [00:20<00:00,  5.36it/s]



results for Colima:
total posts: 108

distribution of posts across dimensions of poverty:
- INCOME: 8 post (7.4%)
- ACCESS TO HEALTH SERVICES: 19 post (17.6%)
- EDUCATIONAL LAG: 6 post (5.6%)
- ACCESS TO SOCIAL SECURITY: 19 post (17.6%)
- HOUSING: 48 post (44.4%)
- ACCESS TO FOOD: 4 post (3.7%)
- SOCIAL COHESION: 4 post (3.7%)
- non-poverty posts: 0 post (0.0%)

analyzing Jalisco (207 posts)...


Classificazione Jalisco: 100%|██████████| 207/207 [01:20<00:00,  2.58it/s]



results for Jalisco:
total posts: 207

distribution of posts across dimensions of poverty:
- INCOME: 25 post (12.1%)
- ACCESS TO HEALTH SERVICES: 16 post (7.7%)
- EDUCATIONAL LAG: 14 post (6.8%)
- ACCESS TO SOCIAL SECURITY: 54 post (26.1%)
- HOUSING: 88 post (42.5%)
- ACCESS TO FOOD: 1 post (0.5%)
- SOCIAL COHESION: 6 post (2.9%)
- non-poverty posts: 3 post (1.4%)

count of posts per dimension:
dimension        ACCESS TO FOOD  ACCESS TO HEALTH SERVICES  \
state                                                        
Baja California               0                          9   
Chihuahua                     6                         21   
Colima                        4                         19   
Guanajuato                    3                         12   
Guerrero                      2                         18   
Jalisco                       1                         16   
Michoacán                     7                         34   
Sinaloa                       0            